In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2

train_setGen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip=True)
test_setGen = ImageDataGenerator(rescale=1./255)

In [26]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='NaH75gQE_msTu_LzsJsrIYdnNNXfkDXVZ_FB_aGygYaj',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'realtimecommunicationmodel-donotdelete-pr-ifadblektavvo7'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [27]:
from io import BytesIO
from zipfile import ZipFile

In [28]:
unzip = ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [29]:
pwd

'/home/wsuser/work'

In [30]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/training_set')

In [31]:
train_set = train_setGen.flow_from_directory('/home/wsuser/work/Dataset/training_set',
                                                  target_size=(128,128),
                                                  class_mode='categorical',
                                                  batch_size=100, color_mode='grayscale')

test_set = test_setGen.flow_from_directory('/home/wsuser/work/Dataset/test_set',
                                                  target_size=(128,128),
                                                  class_mode='categorical',
                                                  batch_size=100, color_mode='grayscale')

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [32]:
cnn_model = Sequential() # Sequential Model

cnn_model.add(Conv2D(128, (3,3), activation='relu', input_shape=(128,128,1))) # Convolution Layer
cnn_model.add(MaxPooling2D(pool_size=(2,2))) # Pooling Layer

cnn_model.add(Conv2D(128, (3,3), activation='relu')) # Convolution Layer
cnn_model.add(MaxPooling2D(pool_size=(2,2))) # Pooling Layer

cnn_model.add(Flatten()) # Flatten Layer
cnn_model.add(Dense(128, activation='relu')) # Hidden Layer
cnn_model.add(Dense(128, activation='relu')) # Hidden Layer
cnn_model.add(Dense(9, activation='softmax')) # Hidden Layer

In [17]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
cnn_model.fit_generator(train_set, 
              steps_per_epoch=len(train_set), 
              epochs=3, 
              validation_data=test_set, 
              validation_steps=len(test_set))

/tmp/wsuser/ipykernel_165/2477806422.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn_model.fit_generator(train_set,


Epoch 1/3
158/158 [==============================] - 1032s 7s/step - loss: 0.2467 - accuracy: 0.9190 - val_loss: 0.2179 - val_accuracy: 0.9698
Epoch 2/3
158/158 [==============================] - 1028s 7s/step - loss: 0.0277 - accuracy: 0.9914 - val_loss: 0.2645 - val_accuracy: 0.9769
Epoch 3/3
158/158 [==============================] - 1031s 7s/step - loss: 0.0145 - accuracy: 0.9952 - val_loss: 0.3736 - val_accuracy: 0.9733


In [19]:
cnn_model.save('trained_cnn_model.h5')

In [20]:
!tar -zcvf image-classification-model.tgz trained_cnn_model.h5

trained_cnn_model.h5


In [21]:
ls -1

Dataset/
image-classification-model.tgz
trained_cnn_model.h5


In [22]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 11.4 MB/s eta 0:00:01


In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "uK6_2BkDeaVcCaMnVSMABbOrpSde4G6uMNgSkXkA55e9"
}
client = APIClient(wml_credentials)

In [24]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [33]:
space_uid = guid_from_space_name(client, 'realtimecommunication')
print("Space UID = " + space_uid)

Space UID = 28bc8022-631a-417f-a9b4-9a242a292066


In [34]:
client.set.default_space(space_uid)

'SUCCESS'

In [35]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [36]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [38]:
model_details = client.repository.store_model(model='image-classification-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id = client.repository.get_model_id(model_details)

In [39]:
model_id

'd69e68d4-2daa-414c-b7a8-95a2ccbe583c'

In [40]:
client.repository.download(model_id, 'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'